Hypothèses communes

* Domaine 1D (x\in[0,L]), (t\in[0,T])
* Équation de la chaleur linéaire
  [
  \partial_t u-\kappa \partial_{xx}u+\alpha u=f(x,t)
  ]
* Paramètres constants (\kappa>0,\ \alpha\ge 0)
* Validation par solution manufacturée quand pertinent
* Discrétisation temporelle : Euler implicite pour DF, solution modale exacte en spectral

---

## CAS 1 — Conditions de Dirichlet homogènes

### Énoncé

[
\begin{cases}
\partial_t u-\kappa u_{xx}+\alpha u=f(x,t), & x\in(0,L),\ t>0\
u(0,t)=u(L,t)=0\
u(x,0)=u_0(x)
\end{cases}
]

---

### Résolution — Différences finies

Grille (x_i=i h,\ i=1,\dots,N,\ h=L/(N+1))

[
\frac{u^{n+1}-u^n}{\Delta t}+\kappa A u^{n+1}+\alpha u^{n+1}=f^{n+1}
]

Matrice (A) : Laplacien 1D Dirichlet
Ordre : (O(h^2+\Delta t))

```python
import numpy as np
from scipy.sparse import spdiags
from scipy.sparse.linalg import spsolve

L, kappa, alpha = 1.0, 1.0, 0.5
N, dt, T = 100, 1e-3, 0.1
h = L/(N+1)
x = np.linspace(h, L-h, N)

u = x*(L-x)
Nt = int(T/dt)

D0 = 2*np.ones(N)/h**2
D1 = -1*np.ones(N-1)/h**2
A = spdiags([D1, D0, D1], [-1,0,1], N, N)

M = np.eye(N) + dt*(kappa*A.toarray()+alpha*np.eye(N))

for _ in range(Nt):
    u = np.linalg.solve(M, u)

```

---

### Résolution — Spectrale (DST)

Base sinus :
[
\phi_k(x)=\sin\left(\frac{k\pi x}{L}\right),\quad
\lambda_k=\left(\frac{k\pi}{L}\right)^2
]

[
\hat u_k(t)=\hat u_k(0)e^{-(\kappa\lambda_k+\alpha)t}
]

```python
from scipy.fftpack import dst, idst

N = 200
x = np.linspace(0, L, N+2)[1:-1]
u0 = x*(L-x)

uhat = dst(u0, type=1)
k = np.arange(1, N+1)
lam = (np.pi*k/L)**2

t = 0.1
uhat_t = uhat*np.exp(-(kappa*lam+alpha)*t)
u = idst(uhat_t, type=1)/(2*N)
```

---

## CAS 2 — Conditions de Neumann homogènes

### Énoncé

[
\begin{cases}
\partial_t u-\kappa u_{xx}=f(x,t)\
u_x(0,t)=u_x(L,t)=0\
u(x,0)=u_0(x)
\end{cases}
]

Condition de compatibilité :
[
\int_0^L f(x,t),dx=0
]

---

### Différences finies

Grille (x_i=(i+\tfrac12)h)

```python
h = L/N
x = (np.arange(N)+0.5)*h
u = np.cos(np.pi*x/L)

D0 = 2*np.ones(N)/h**2
D0[0] = D0[-1] = 1/h**2
D1 = -1*np.ones(N-1)/h**2
A = spdiags([D1, D0, D1], [-1,0,1], N, N)

for _ in range(Nt):
    u = spsolve(np.eye(N)+dt*kappa*A, u)
```

---

### Spectrale — DCT

[
\phi_k(x)=\cos\left(\frac{k\pi x}{L}\right)
]

```python
from scipy.fftpack import dct, idct

u0 = np.cos(np.pi*x/L)
uhat = dct(u0, type=2)
k = np.arange(N)
lam = (np.pi*k/L)**2
lam[0] = 0

t = 0.1
uhat_t = uhat*np.exp(-kappa*lam*t)
u = idct(uhat_t, type=2)/(2*N)
```

---

## CAS 3 — Conditions périodiques

### Énoncé

[
\begin{cases}
\partial_t u-\kappa u_{xx}+\alpha u=f(x,t)\
u(0,t)=u(L,t),\quad u_x(0,t)=u_x(L,t)\
u(x,0)=u_0(x)
\end{cases}
]

---

### Différences finies périodiques

Matrice circulante

```python
x = np.linspace(0, L, N, endpoint=False)
u = np.sin(2*np.pi*x/L)

D0 = 2*np.ones(N)/h**2
D1 = -1*np.ones(N)/h**2
A = spdiags([D1, D0, D1], [-1,0,1], N, N).toarray()
A[0,-1] = A[-1,0] = -1/h**2

for _ in range(Nt):
    u = np.linalg.solve(np.eye(N)+dt*(kappa*A+alpha*np.eye(N)), u)
```

---

### Spectrale — FFT réelle

[
\lambda_k=(2\pi k/L)^2
]

```python
from scipy.fft import rfft, irfft

x = np.linspace(0, L, N, endpoint=False)
u0 = np.sin(2*np.pi*x/L)

uhat = rfft(u0)
k = np.arange(0, N//2+1)
lam = (2*np.pi*k/L)**2

t = 0.1
uhat_t = uhat*np.exp(-(kappa*lam+alpha)*t)
u = irfft(uhat_t)
```

---

## CAS 4 — Chaleur avec source dépendant du temps (Duhamel)

### Énoncé

[
\partial_t u-\kappa u_{xx}=f(x,t)
]

---

### Spectrale (solution exacte modale)

[
\hat u_k(t)=e^{-\kappa\lambda_k t}\hat u_k(0)
+\int_0^t e^{-\kappa\lambda_k(t-s)}\hat f_k(s),ds
]

Approximation rectangle :

```python
uhat = rfft(u0)
for n in range(Nt):
    t = n*dt
    fhat = rfft(f(x,t))
    uhat = np.exp(-kappa*lam*dt)*uhat + dt*fhat
u = irfft(uhat)
```

